
# 📝 SportsCrew · **Ollama (ejercicio para alumnos)**
Completa las celdas marcadas con **TODO**. Hay **pistas** en los comentarios de cada celda.

**Objetivo**: generar un *artículo breve* y **5 tweets** sobre una liga deportiva usando **Ollama (mistral)** con **CrewAI**.

**Documentación útil**
- **Ollama (oficial):** https://github.com/ollama/ollama  
- **CrewAI (conceptos):** https://docs.crewai.com/concepts/  
- **CrewAI (API referencia):** https://docs.crewai.com/reference/  
- **LiteLLM (enrutador):** https://docs.litellm.ai/  
- **LangChain (ecosistema):** https://python.langchain.com/docs/  
- **DuckDuckGo (herramienta LC):** https://python.langchain.com/docs/integrations/tools/ddg  
- **Requests (HTTP):** https://requests.readthedocs.io/  
- **python-dotenv:** https://github.com/theskumar/python-dotenv  
- **json (stdlib):** https://docs.python.org/3/library/json.html




## 1) Instalar dependencias en este kernel
Rellena la lista `packages` con las librerías necesarias y completa la llamada a `pip install`.

**Pistas**:
- Necesitarás: `crewai`, `litellm`, `langchain`, `langchain-community`, `langchain-openai`, `duckduckgo-search`, `requests`, `python-dotenv`.
- Usa `sys.executable -m pip install -U ...` para asegurar instalación en este kernel.


In [1]:
import importlib, sys, subprocess
packages = [
    "crewai>=0.63.6",
    "litellm>=1.40.11",
    "langchain>=0.2.10",
    "langchain-community>=0.2.10",
    "langchain-openai",
    "duckduckgo-search",
    "requests",
    "python-dotenv"
]
to_install = []
for spec in packages:
    mod = spec.split("==")[0].split(">=")[0].replace("-", "_")
    try:
        importlib.import_module(mod if mod!="python_dotenv" else "dotenv")
    except Exception:
        to_install.append(spec)
if to_install:
    print("Instalando:", to_install)
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", *to_install])
else:
    print("Todo estaba instalado ✅")


Todo estaba instalado ✅



## 2) Comprobar **Ollama** y el modelo `mistral`
Debes verificar que:
1) El binario `ollama` está en el PATH.
2) La API local responde en `http://localhost:11434/api/tags`.
3) El modelo `mistral` está descargado.

**Pistas**:
- Usa `shutil.which("ollama")` para comprobar el binario.
- Usa `requests.get(url, timeout=5)` y revisa `resp.ok`.
- La respuesta JSON tiene una clave `models` con nombres; busca `"mistral"`.


In [2]:


import shutil, requests
print("Binario 'ollama' en PATH:", "✅" if shutil.which("ollama") else "❌")
try:
    r = requests.get("http://localhost:11434/api/tags", timeout=5)
    if r.ok:
        tags = [m.get("name") for m in r.json().get("models", []) if isinstance(m, dict)]
        print("Ollama API:", "✅ OK")
        print("Modelos locales:", tags)
        print("'mistral' presente:", "✅" if any("mistral" in (t or "") for t in tags) else "❌ (ejecuta: ollama pull mistral)")
    else:
        print("Ollama API respondió con error HTTP:", r.status_code)
except Exception as e:
    print("No se pudo conectar a http://localhost:11434. ¿Ejecutaste 'ollama serve'?")
    print("Detalle:", e)


Binario 'ollama' en PATH: ✅
Ollama API: ✅ OK
Modelos locales: ['mistral:instruct', 'mistral:latest']
'mistral' presente: ✅




## 3) Elegir deporte y liga
Cambia estos valores si quieres otra competición. Ejemplos:
1. **soccer**: 
- `esp.1` (LaLiga)
- `eng.1` (Premier)
- `ita.1` (Calcio)
- `fra.1` (ligue 1) 
- `usa.1` (MLS)
2. **basketball**: 
- `nba`
- `wnba`
3. **football**: 
- `nfl` 
4. **baseball**: 
- `mlb` 
5. **hockey**: 
- `nhl`



In [3]:

SPORT = "soccer"
LEAGUE = "esp.1"
MAX_ARTICLES = 5
print("Deporte:", SPORT, "| Liga:", LEAGUE)


Deporte: soccer | Liga: esp.1



## 4) Descargar noticias (ESPN)
Construye la URL y parsea el JSON. Guarda los artículos en `articles` (máximo `MAX_ARTICLES`).

**Pistas**:
- URL base: `http://site.api.espn.com/apis/site/v2/sports/{sport}/{league}/news`
- A veces los datos están en `data["articles"]`; otras en `data["feed"]["entries"]`.
- Para `entries`, normaliza a `{"headline": ...}`.


In [4]:
import requests

def get_news_url(sport, league):
    """
    Genera la URL para obtener noticias de ESPN según el deporte y la liga.

    Args:
        sport (str): Deporte (por ejemplo, "soccer").
        league (str): Liga específica (por ejemplo, "esp.1" para LaLiga).

    Returns:
        str: URL para obtener las noticias.
    """
    return f"http://site.api.espn.com/apis/site/v2/sports/{sport}/{league}/news"

# Construye la URL para el deporte y la liga seleccionados.
url = get_news_url(SPORT, LEAGUE)

# Realiza una solicitud HTTP para obtener las noticias.
resp = requests.get(url, timeout=15)
resp.raise_for_status()  # Lanza una excepción si la solicitud falla.

# Procesa la respuesta JSON.
data = resp.json()
articles = []  # Lista para almacenar los artículos.

# Extrae los artículos de la respuesta si el formato es válido.
if isinstance(data, dict):
    if "articles" in data and isinstance(data["articles"], list):
        # Si la clave "articles" está presente, utiliza su contenido.
        articles = data["articles"]
    elif "feed" in data and isinstance(data["feed"], dict):
        # Si la clave "feed" está presente, extrae los titulares de las entradas.
        entries = data["feed"].get("entries", [])
        articles = [{"headline": e.get("headline") or e.get("title", "")} for e in entries]

# Imprime la cantidad de artículos encontrados.
print(f"Artículos encontrados: {len(articles)}")

# Si no se encontraron artículos, muestra un mensaje.
if not articles:
    print("No hay noticias ahora mismo. Prueba otra liga o vuelve más tarde.")

# Limita el número de artículos a procesar según MAX_ARTICLES.
articles = articles[:MAX_ARTICLES]


Artículos encontrados: 6



## 5) Búsqueda rápida (opcional) con DuckDuckGo
Intenta crear un wrapper y un `run` de búsqueda para obtener contexto por titular.

In [5]:
try:
    from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
    try:
        from langchain_community.tools import DuckDuckGoSearchRun
    except Exception:
        from langchain.tools import DuckDuckGoSearchRun
    ddg = DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region="us-en", time="d"))
except Exception as e:
    ddg = None
    print("DuckDuckGo no disponible, seguimos sin búsqueda. Detalle:", e)



## 6) Configurar el modelo (Ollama · mistral) para CrewAI
Crea un `LLM` de CrewAI que apunte a Ollama.


In [6]:
from crewai import Agent, Task, Crew, LLM

# Configura el modelo de lenguaje LLM utilizando Ollama.
llm = LLM(
    model="ollama/mistral",  # Especifica el modelo local "mistral" de Ollama.
    base_url="http://localhost:11434",  # URL base para conectarse al servidor de Ollama.
    temperature=0.2,  # Controla la aleatoriedad en las respuestas generadas.
    api_key="ollama-local"  # Clave API para autenticar el uso del modelo local.
)

# Imprime un mensaje indicando que el modelo está listo.
print("LLM listo ✅")


LLM listo ✅



## 7) Resúmenes (120–180 palabras)
Para cada artículo, crea un agente **Investigador** y genera un resumen.

**Pistas**:
- Importa: `from crewai import Agent, Task, Crew`
- `headline = art.get("headline") or art.get("title") or ""`
- Si `ddg` existe y hay `headline`, llama a `ddg.run(headline)` (maneja excepciones).
- Prompt: “Resume en 120–180 palabras. SOLO el resumen...” (añade titular y contexto).

Guarda cada texto en la lista `summaries`.


In [7]:
summaries = []  # Lista para almacenar los resúmenes generados.
for i, art in enumerate(articles, start=1):  # Itera sobre los artículos descargados.
    headline = art.get("headline") or art.get("title") or ""  # Obtiene el titular del artículo.
    ctx = ""  # Contexto adicional (vacío por defecto).
    if ddg and headline:  # Si DuckDuckGo está disponible y hay un titular:
        try:
            ctx = ddg.run(headline)  # Realiza una búsqueda rápida para obtener contexto adicional.
        except Exception:
            ctx = ""  # Si ocurre un error, deja el contexto vacío.
    
    # Configura un agente con el rol de "Investigador" para generar resúmenes.
    researcher = Agent(
        role="Investigador",  # Rol del agente.
        goal="Resumen claro y corto en español.",  # Objetivo del agente.
        backstory="Resumes rápido.",  # Contexto adicional del agente.
        llm=llm,  # Modelo de lenguaje configurado.
        allow_delegation=False  # No permite delegar tareas.
    )
    
    # Define el prompt para el agente, incluyendo el titular y el contexto.
    prompt = f"""Resume en 120–180 palabras. SOLO el resumen (sin título ni listas).
Titular: {headline}
Contexto (puede estar vacío):
{ctx}
"""
    # Crea una tarea para el agente con el prompt y el formato esperado del resultado.
    task = Task(
        agent=researcher,  # Agente encargado de la tarea.
        description=prompt,  # Descripción de la tarea.
        expected_output="Resumen en español, 120–180 palabras."  # Formato esperado del resultado.
    )
    
    # Ejecuta la tarea utilizando CrewAI y obtiene el resultado.
    result = Crew(agents=[researcher], tasks=[task], verbose=0).kickoff()
    text = str(getattr(result, "raw_output", result))  # Extrae el texto del resultado.
    summaries.append(text.strip())  # Agrega el resumen a la lista, eliminando espacios innecesarios.

# Imprime la cantidad de resúmenes generados.
print(f"Resúmenes creados: {len(summaries)}")



Resúmenes creados: 5



## 8) Artículo (200–250 palabras, 2–3 párrafos)
Crea un agente **Periodista** y genera el artículo usando los resúmenes.

**Pistas**:
- Une `summaries` con `\n\n` → `summaries_text`.
- Agent con `role="Periodista Deportivo"` y `llm=llm`.
- Prompt claro: “Escribe un artículo de 200–250 palabras (2–3 párrafos)...”
- Ejecuta `Crew(...).kickoff()` y guarda en `article_text`.


In [8]:
summaries_text = "\n\n".join(summaries) if summaries else "Sin resúmenes."
# Combina los resúmenes generados en un solo texto, separados por dos saltos de línea.
# Si no hay resúmenes disponibles, utiliza el texto "Sin resúmenes."

# Configura un agente con el rol de "Periodista" para generar un artículo breve.
journalist = Agent(
    role="Periodista",  # Rol del agente.
    goal="Artículo breve y claro en español.",  # Objetivo del agente.
    backstory="Escribes con precisión.",  # Contexto adicional del agente.
    llm=llm,  # Modelo de lenguaje configurado.
    allow_delegation=False  # No permite delegar tareas a otros agentes.
)

# Define el prompt para el agente, incluyendo los resúmenes como base para el artículo.
article_prompt = f"""Escribe un artículo en español de 200–250 palabras (2–3 párrafos). Sin listas ni encabezados.
Basado en:
{summaries_text}
"""

# Crea una tarea para el agente con el prompt y el formato esperado del resultado.
article_task = Task(
    agent=journalist,  # Agente encargado de la tarea.
    description=article_prompt,  # Descripción de la tarea.
    expected_output="Artículo breve en español (200–250 palabras)."  # Formato esperado del resultado.
)

# Ejecuta la tarea utilizando CrewAI y obtiene el resultado.
article_text = str(Crew(agents=[journalist], tasks=[article_task], verbose=0).kickoff())

# Imprime el artículo generado.
print("Artículo listo ✅\n")
print(article_text)



Artículo listo ✅

Atlético Madrid consigue su segundo triunfo de la temporada en LaLiga tras derrotar por 3-2 al Rayo Vallecano, gracias al hat-trick espectacular de Julián Álvarez. El delantero argentino ha sido elogiado por Diego Simeone como "el mejor jugador que tenemos", lo que garantizó el lugar de Atlético en las semifinales. Sin embargo, la negociación con el jugador llegó a un punto tan tensa que no fue convocado para el partido contra Atlético Tucumán el pasado día.

En los últimos 15 encuentros entre Atletico Madrid y Rayo Vallecano, el primer equipo ha ganado en 12 ocasiones, mientras que Rayo Vallecano no ha obtenido victorias. El partido más reciente terminó con un marcador de 3-2 a favor de Atletico Madrid. Destacan los goles de Julián Álvarez y Fran Pérez por el equipo local. Barcelona también se enfrentó a Rayo Vallecano, ganando 1-2 gracias a un penal convertido por Lamine Yamal antes del tiempo de descanso, pero una gran anotación de Fran Pérez empató el partido.

En


## 9) 5 tweets (1 línea c/u, con emojis y 1–2 hashtags)
Crea un agente **Creador de Tweets** y genera EXACTAMENTE 5 líneas numeradas (1–5).

**Pistas**:
- Cada tweet debe referirse a un **dato distinto** del artículo.
- Prompt: “Crea EXACTAMENTE 5 tweets numerados (1–5)... máx 40 palabras, 1–2 hashtags, emojis.”
- Muestra las 5 líneas resultantes.


In [9]:
influencer = Agent(
    role="Creador de Tweets",  # Rol del agente.
    goal="Escribir 5 tweets en una línea.",  # Objetivo del agente.
    backstory="Experto en X.",  # Contexto adicional del agente.
    llm=llm,  # Modelo de lenguaje configurado.
    allow_delegation=False  # No permite delegar tareas.
)

# Define el prompt para el agente, incluyendo las reglas y el artículo como base.
tweets_prompt = f"""Crea EXACTAMENTE 5 tweets numerados (1–5), cada uno en UNA línea, en español.
Reglas: cada tweet debe referirse a un dato distinto del ARTÍCULO, incluir 1–2 hashtags y emojis, máx 40 palabras.
ARTÍCULO:
{article_text}
"""

# Crea una tarea para el agente con el prompt y el formato esperado del resultado.
tweets_task = Task(
    agent=influencer,  # Agente encargado de la tarea.
    description=tweets_prompt,  # Descripción de la tarea.
    expected_output="5 líneas numeradas 1–5."  # Formato esperado del resultado.
)

# Ejecuta la tarea utilizando CrewAI y obtiene el resultado.
tweets_text = str(Crew(agents=[influencer], tasks=[tweets_task], verbose=0).kickoff())

# Procesa el texto generado para extraer las líneas de los tweets.
lines = [ln.strip() for ln in tweets_text.splitlines() if ln.strip()]

# Verifica si se generaron exactamente 5 tweets.
if len(lines) < 5:
    print("Resultado recibido:")
    print(tweets_text)
    print("\nNo se detectaron 5 tweets claros. Ejecuta de nuevo esta celda.")
else:
    print("\n================= 🐦 Tweets =================\n")
    for ln in lines[:5]:  # Imprime los primeros 5 tweets generados.
        print(ln)



================= 🐦 Tweets =================

1. 🏆 Atlético Madrid gana 3-2 contra Rayo Vallecano gracias al hat-trick de Julián Álvarez (#Atleti #LaLiga)
2. 🔥 Alargue su récord a 12 victorias en los últimos 15 encuentros frente a Rayo Vallecano (#AtletiRecord)
3. 🏃‍♂️ Atlético avanza a semifinales tras elogio de Diego Simeone sobre Julián Álvarez (#Simeone #Alvarez)
4. 💔 No convocado para el partido contra Atlético Tucumán debido a negociaciones tensas (#AtletiNegociaciones)
5. 🏡 Propiedades en venta: Jefferson City, MO ($179,900), Lancaster, OH ($279,950) y Carson City, NV ($599,000). Busca propiedades recientemente vendidas en Realtor.com® (#Propiedades #Venta)
